# Performance of different unsupervised anomaly detectors  across different datasets
#### Writer: Hamidreza Salahi
#### Date: July 2024

## 1. Goal and methods

This study aims to evaluate the performance of various unsupervised anomaly detection algorithms across multiple datasets. Using the Python Outlier Detection (PyOD) library, we will compare the following models:

- K-Nearest Neighbors (KNN)
- Local Outlier Factor (LOF)
- Isolation Forest (IForest)
- Kernel Density Estimation (KDE)
- Principal Component Analysis (PCA)

The datasets selected for this analysis are sourced from the ADbench repository, which includes:

- annthyroid
- campaign
- census
- donors
- skin

The evaluation metric used is the Area Under the Receiver Operating Characteristic Curve (AUC-ROC), with hyperparameters optimized through grid search and cross-validation. Given the large size of these datasets, subsampling with stratification on the target variable $\textbf{y}$ is employed to maintain the class balance between inliers and outliers. This comprehensive evaluation will provide insights into the effectiveness of different anomaly detection techniques across diverse data scenarios.

In [26]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from pyod.models.knn import KNN  
from pyod.models.lof import LOF 
from pyod.models.iforest import IForest 
from pyod.models.pca import PCA 
from pyod.models.kde import KDE 
import itertools
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

## 2. Load datasets and subsampling

In [27]:
# Load a dataset from ADbench
adbench_dir = '../../anaconda3/lib/python3.10/site-packages/adbench/datasets/Classical/' 
dataset_names = ['2_annthyroid', '5_campaign', '9_census', '11_donors','33_skin' ]  
outliers_dict = {}
for dataset in dataset_names:
    data = np.load(adbench_dir+dataset+'.npz', allow_pickle=True)
    dataset_name = dataset.split("_")[1].split(".")[0]
    outliers = round(sum(data["y"]/len(data["y"])*100),2)
    outliers_dict[dataset_name] = outliers
    print(f'{dataset_name} dataset has {len(data["y"])} rows, {data["X"].shape[1]} features,'
          f' mean:{np.mean(data["X"])}, standard deviation:{np.std(data["X"])} and {outliers}% outliers')
    

annthyroid dataset has 7200 rows, 6 features, mean:0.14430651111111112, standard deviation:0.19154795592282345 and 7.42% outliers
campaign dataset has 41188 rows, 62 features, mean:0.22784682179745439, standard deviation:0.40090420621193684 and 11.27% outliers
census dataset has 299285 rows, 500 features, mean:0.06630965663919679, standard deviation:0.24769828770621777 and 6.2% outliers
donors dataset has 619326 rows, 10 features, mean:0.3276509663837787, standard deviation:0.4354913258213057 and 5.93% outliers
skin dataset has 245057 rows, 3 features, mean:-1.3855743303063907e-16, standard deviation:0.9999979596562837 and 20.75% outliers


### note:
None of the datasets are standardized (mean = 0, stdv = 1) except for skin dataset. The models which use a distance metric such as PCA, KNN and LOF require scaling. I will apply StandardScaler separately to training and test data to prevent data leakage when it comes to data modeling.

   Due to the limitation of computation power, I will be exploiting subsampling, keeping only 1000 data points from each dataset and saving the new subsamples as a dictionary with keys being the name of the dataset and values being $\textbf{X,y}$

In [28]:
# Number of rows in the subsample
n_subsample = 1000

In [29]:
# Dictionary to hold the data
datasets = {}

for dataset in dataset_names:
    data = np.load(adbench_dir + dataset+'.npz', allow_pickle=True)
    X = data['X']
    y = data['y']
    data_combined = np.hstack((X, y.reshape(-1, 1)))  # Combine X and y to keep them together during sampling
    subsampled_data = resample(data_combined, n_samples=n_subsample, stratify=y, random_state=42)  # Subsample the data while maintaining the class distribution
    # Separate the subsample into features and target variable
    X_subsample = subsampled_data[:, :-1]
    y_subsample = subsampled_data[:, -1]
    # Store the subsampled data
    datasets[dataset] = (X_subsample, y_subsample)

In [30]:
# sanity check for subsample with regards to outliers
for dataset, (X, y) in datasets.items():
    dataset_name = dataset.split("_")[1].split(".")[0]
    print(f'{dataset_name} subsamplle has {len(y)} rows, {X.shape[1]} features and {round(sum(y/len(y)*100),2)}% outliers, actual {dataset_name} dataset has {outliers_dict[dataset_name]}% outliers ')

annthyroid subsamplle has 1000 rows, 6 features and 7.4% outliers, actual annthyroid dataset has 7.42% outliers 
campaign subsamplle has 1000 rows, 62 features and 11.3% outliers, actual campaign dataset has 11.27% outliers 
census subsamplle has 1000 rows, 500 features and 6.2% outliers, actual census dataset has 6.2% outliers 
donors subsamplle has 1000 rows, 10 features and 5.9% outliers, actual donors dataset has 5.93% outliers 
skin subsamplle has 1000 rows, 3 features and 20.8% outliers, actual skin dataset has 20.75% outliers 


 The % of the outliers in the subsample data is consistent with the actual dataset

## 3. Modeling, Hyperparameters optimization and storing the best model across the datasets

For each dataset, multiple unsupervised anomaly detection models are trained using the PyOD library. Hyperparameters for each model are optimized through manually performed grid search and cross-validation, as GridSearchCV is not consistent with unsupervised learning and ROC AUC scoring. After identifying the best set of hyperparameters, we evaluate each model using the AUC-ROC metric. The best-performing model for each dataset is saved for future analysis and comparison.

In [31]:
# Define the anomaly detection models and their hyperparameters
models = {
   
    'KNN': {
        'model': KNN,
        'params': {
            'n_neighbors': [3, 5, 10, 15],
            'method': ['largest', 'mean', 'median'],
            'contamination': [0.05, 0.1, 0.15, 0.2]
        }
    },
        'LOF': {
        'model': LOF,
        'params': {
          'n_neighbors': [5, 10,20,30],
            'contamination': [0.05, 0.1, 0.15, 0.2]

                }
    },
    'IForest': {
        'model': IForest,
        'params': {
            'n_estimators': [50, 75, 100, 150],
            'max_features': [0.05, 0.1, 0.15, 0.2],
            'contamination': [0.05, 0.1, 0.15, 0.2]
        }
    },
    'PCA': {
        'model': PCA,
        'params': {
            
            'whiten': [True, False],
            'contamination': [0.05, 0.1, 0.15, 0.2]
        }
    },
    'KDE': {
        'model': KDE,
        'params': {
            'leaf_size': [10, 20, 30, 40],
            'contamination': [0.05, 0.1, 0.15, 0.2]
           
        }
    }
    
}

In [32]:
# Evaluate the models
results = []
        
for dataset, (X, y) in datasets.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    # Apply StandardScaler to training data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform test data using the same scaler
    X_test_scaled = scaler.transform(X_test)
    for model_name, model_info in models.items():
        model_class = model_info['model']  # This should be the class, not an instance
        param_combinations = list(itertools.product(*(model_info['params'][param] for param in model_info['params'])))
        
        best_auc_roc = 0
        best_model = None
        best_params = None
        
        for param_combination in param_combinations:
            params = {param: value for param, value in zip(model_info['params'].keys(), param_combination)}
            model = model_class(**params)
            
             # Manual cross-validation
            skf = StratifiedKFold(n_splits=5)
            auc_scores = []
            
            for train_index, val_index in skf.split(X_train_scaled, y_train):
                X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[val_index]
                y_train_cv, y_val_cv = y_train[train_index], y_train[val_index]
                
                model.fit(X_train_cv)
                y_pred = model.decision_function(X_val_cv)
                
                auc_roc = roc_auc_score(y_val_cv, y_pred)
                
                auc_scores.append(auc_roc)
            
            avg_auc_roc = np.mean(auc_scores)
            
            # Update best model if the current one is better
            if avg_auc_roc > best_auc_roc:
                best_auc_roc = avg_auc_roc
                best_model = model
                best_params = params
        y_pred_test = best_model.decision_function(X_test_scaled)
        test_auc_roc = roc_auc_score(y_test, y_pred_test)
        # Store the best model results
        results.append({
            'dataset': dataset.split("_")[1].split(".")[0],
            'model': model_name,
            'best_params': best_params,
            'best_auc_roc': best_auc_roc,
            'test_auc_roc': test_auc_roc,
            'best_model': best_model
        })


## 4. Display the results

In [33]:
# Create lists to hold data for DataFrame construction
datasets_list = []
models_list = []
auc_rocs_lists = []

# Populate lists with results
for result in results:
    datasets_list.append(result['dataset'])
    models_list.append(result['model'])
    auc_rocs_lists.append(result['best_auc_roc'])

# Create a dictionary to construct the DataFrame
data_dict = {
    'dataset': datasets_list,
    'model': models_list,
    'best_auc_roc': auc_rocs_lists
}

# Create the DataFrame
results_df = pd.DataFrame(data_dict)

# Pivot the DataFrame to have models as columns
results_df = results_df.pivot_table(index=['dataset'], columns='model', values='best_auc_roc').reset_index()

# Highlight the highest value in each row
def highlight_max_row(row):
    # Find the maximum value in the row
    max_val = row.max()
    # Create a Series to hold the styles
    styles = ['' if v != max_val else 'font-weight: bold' for v in row]
    return styles

# Apply the highlight_max_row function row-wise
results_df = results_df.set_index(['dataset'])  # Set the index for row-wise operation
results_df = results_df.style.apply(highlight_max_row, axis=1).format(precision=4)

# Display the styled DataFrame
results_df


model,IForest,KDE,KNN,LOF,PCA
dataset,,,,,
annthyroid,0.8963,0.6524,0.6931,0.6735,0.6372
campaign,0.7437,0.6639,0.7329,0.7106,0.7177
census,0.6742,0.6715,0.6850,0.5915,0.6593
donors,0.8608,0.7904,0.8236,0.6062,0.7703
skin,0.5924,0.6502,0.7686,0.3726,0.4001


In [34]:
# Save the styled DataFrame to an Excel file
results_df.to_excel('anomaly_detection_results.xlsx', engine='openpyxl')

## 5. Experiment with outlier ratio

An intriguing experiment involves varying the outlier ratio in each dataset to analyze model performance as a function of this ratio. To accomplish this, I first create subsamples of each dataset with outlier ratios of 5%, 10%, 15%, and 20%. Using the same methodology as previously employed (grid search and cross-validation), I determine the optimal hyperparameters for each model. This approach allows us to evaluate and compare model performance across different outlier ratios effectively.

In [42]:
# Define desired outlier percentages
outlier_percentages = [0.05, 0.10, 0.15, 0.2]


# Dictionary to hold the data
datasets = {perc: {} for perc in outlier_percentages}

for dataset in dataset_names:
    data = np.load(adbench_dir + dataset + '.npz', allow_pickle=True)
    X = data['X']
    y = data['y']
    
    # Identify inliers and outliers
    inliers = X[y == 0]
    outliers = X[y == 1]
    
    for perc in outlier_percentages:
        # Calculate the number of inliers and outliers needed
        n_outliers = int(n_subsample * perc)
        n_inliers = n_subsample - n_outliers
        
        # Resample inliers and outliers separately
        resampled_inliers = resample(inliers, n_samples=n_inliers, random_state=42)
        resampled_outliers = resample(outliers, n_samples=n_outliers, random_state=42)
        
        # Combine resampled inliers and outliers
        X_subsample = np.vstack((resampled_inliers, resampled_outliers))
        y_subsample = np.hstack((np.zeros(n_inliers), np.ones(n_outliers)))
        
        # Shuffle the combined data to mix inliers and outliers
        combined_data = np.hstack((X_subsample, y_subsample.reshape(-1, 1)))
        np.random.shuffle(combined_data)
        
        # Split back into features and target
        X_subsample = combined_data[:, :-1]
        y_subsample = combined_data[:, -1]
        
        # Store the subsampled data
        datasets[perc][dataset] = (X_subsample, y_subsample)

In [49]:
# Evaluate the models
results = []
for perc, dataset_dict in datasets.items():
    for dataset, (X, y) in dataset_dict.items():        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
        # Apply StandardScaler to training data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        # Transform test data using the same scaler
        X_test_scaled = scaler.transform(X_test)
        for model_name, model_info in models.items():
            model_class = model_info['model']  # This should be the class, not an instance
            param_combinations = list(itertools.product(*(model_info['params'][param] for param in model_info['params'])))

            best_auc_roc = 0
            best_model = None
            best_params = None

            for param_combination in param_combinations:
                params = {param: value for param, value in zip(model_info['params'].keys(), param_combination)}
                model = model_class(**params)

                 # Manual cross-validation
                skf = StratifiedKFold(n_splits=5)
                auc_scores = []

                for train_index, val_index in skf.split(X_train_scaled, y_train):
                    X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[val_index]
                    y_train_cv, y_val_cv = y_train[train_index], y_train[val_index]

                    model.fit(X_train_cv)
                    y_pred = model.decision_function(X_val_cv)

                    auc_roc = roc_auc_score(y_val_cv, y_pred)

                    auc_scores.append(auc_roc)

                avg_auc_roc = np.mean(auc_scores)

                # Update best model if the current one is better
                if avg_auc_roc > best_auc_roc:
                    best_auc_roc = avg_auc_roc
                    best_model = model
                    best_params = params
            y_pred_test = best_model.decision_function(X_test_scaled)
            test_auc_roc = roc_auc_score(y_test, y_pred_test)
            # Store the best model results
            results.append({
                'dataset': dataset.split("_")[1].split(".")[0],
                'model': model_name,
                'outlier_percentage': perc,
                'best_params': best_params,
                'best_auc_roc': best_auc_roc,
                'test_auc_roc': test_auc_roc,
                'best_model': best_model
            })


In [ ]:
# Create lists to hold data for DataFrame construction
datasets_list = []
outlier_percentages = []
models_list = []
auc_rocs_list = []

# Populate lists with results
for result in results:
    datasets_list.append(result['dataset'])
    outlier_percentages.append(result['outlier_percentage']*100)
    models_list.append(result['model'])
    auc_rocs_list.append(result['test_auc_roc'])

# Create a dictionary to construct the DataFrame
data_dict = {
    'dataset': datasets_list,
    'outlier_percentage': outlier_percentages,
    'model': models_list,
    'test_auc_roc': auc_rocs_list
}

# Create the DataFrame
results_df = pd.DataFrame(data_dict)

# Pivot the DataFrame to have models as columns
results_df = results_df.pivot_table(index=['dataset', 'outlier_percentage'], columns='model', values='test_auc_roc').reset_index()
# Highlight the highest value in each row
def highlight_max_row(row):
    # Find the maximum value in the row
    max_val = row.max()
    # Create a Series to hold the styles
    styles = ['' if v != max_val else 'font-weight: bold' for v in row]
    return styles

# Apply the highlight_max_row function row-wise
results_df = results_df.set_index(['dataset', 'outlier_percentage'])  # Set the index for row-wise operation
results_df = results_df.style.apply(highlight_max_row, axis=1).format(precision=4)

# Save the styled DataFrame to an Excel file
results_df.to_excel('anomaly_detection_results_different_outlier.xlsx', engine='openpyxl')
results_df